In [49]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")

   Updating registry at `C:\Users\kgeri\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  99.9 %Fetching: [====>                                    ]  8.3 % %Fetching: [===============>                         ]  35.9 %40.8 %Fetching: [===================>                     ]  45.8 %]  50.4 %]  66.6 %Fetching: [==============================>          ]  74.4 %=================================>       ]  80.4 % %.9 %

  Resolving package versions...
  Installed Zlib_jll ──────── v1.2.11+18
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed OpenSpecFun_jll ─ v0.5.3+4
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Project.toml`
Updating `C:\Users\kgeri\.julia\environments\v1.5\Manifest.toml`
  [efe28fd5] ↑ OpenSpecFun_jll v0.5.3+3 ⇒ v0.5.3+4
  [8f1865be] ↑ ZeroMQ_jll v4.3.2+4 ⇒ v4.3.2+5
  [83775a58] ↑ Zlib_jll v1.2.11+17 ⇒ v1.2.11+18
  Resolving package versions...
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Manifest.toml`


In [145]:
using DataFrames
using CSV
using Printf
using Statistics

In [146]:
using PorousMaterials
using LightGraphs
using GraphPlot

In [147]:
pwd()

"C:\\Users\\kgeri"

In [148]:
atoms = read_xyz(joinpath("Documents", "Grad School", "Research", "NewTest.xyz"))

#atoms.species
#atoms.coords

Atoms{Cart}(29, [:O, :C, :O, :C, :H, :C, :C, :N, :N, :C  …  :C, :C, :C, :H, :H, :H, :H, :O, :O, :H], Cart([1.8979 0.655 … 2.8537 0.9385; 3.4866 4.0003 … 13.3672 14.818; 0.8982 0.7805 … 0.1986 0.7708]))

In [149]:
box = Box(15.0,15.0,15.0)
atoms=Frac(atoms, box)

Atoms{Frac}(29, [:O, :C, :O, :C, :H, :C, :C, :N, :N, :C  …  :C, :C, :C, :H, :H, :H, :H, :O, :O, :H], Frac([0.12652666666666662 0.04366666666666665 … 0.1902466666666666 0.0625666666666666; 0.23244 0.2666866666666667 … 0.8911466666666666 0.9878666666666667; 0.059879999999999996 0.05203333333333333 … 0.01324 0.05138666666666667]))

In [150]:
crystal = Crystal("NewTest.cif", box, atoms, Charges{Frac}(0))

Name: NewTest.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 15.000000 Å. b = 15.000000 Å, c = 15.000000 Å
	Volume of unit cell: 3375.000000 Å³

	# atoms = 29
	# charges = 0
	chemical formula: Dict(:N => 2,:H => 12,:O => 4,:C => 11)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [151]:
infer_bonds!(crystal, false)

In [152]:
Graph(crystal.bonds)
#Graph(edges(crystal.bonds))

{29, 30} undirected simple Int64 graph

In [153]:
#typeof(crystal.bonds)
#edges(crystal.bonds)

In [154]:
for ed in edges(crystal.bonds)
   
    println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = crystal.atoms.species[i]
    species_j = crystal.atoms.species[j]
    println("atom ", species_i, " is connected to ", species_j)
    

end

Edge 1 => 2
atom O is connected to C
Edge 1 => 5
atom O is connected to H
Edge 2 => 3
atom C is connected to O
Edge 2 => 4
atom C is connected to C
Edge 4 => 6
atom C is connected to C
Edge 4 => 9
atom C is connected to N
Edge 4 => 16
atom C is connected to H
Edge 6 => 7
atom C is connected to C
Edge 6 => 13
atom C is connected to H
Edge 6 => 15
atom C is connected to H
Edge 7 => 8
atom C is connected to N
Edge 7 => 10
atom C is connected to C
Edge 7 => 14
atom C is connected to H
Edge 8 => 9
atom N is connected to N
Edge 8 => 12
atom N is connected to H
Edge 9 => 11
atom N is connected to H
Edge 10 => 17
atom C is connected to C
Edge 10 => 21
atom C is connected to C
Edge 17 => 18
atom C is connected to C
Edge 17 => 24
atom C is connected to H
Edge 18 => 19
atom C is connected to C
Edge 18 => 23
atom C is connected to H
Edge 19 => 20
atom C is connected to C
Edge 19 => 22
atom C is connected to C
Edge 20 => 21
atom C is connected to C
Edge 20 => 26
atom C is connected to H
Edge 21 => 

In [155]:
#crystal.atoms.species[2]
#nbs = neighbors(crystal.bonds, 1)
crystal.atoms.species[2]

:C

In [156]:
### Function to identify carboxyl. When species is carbon, checks if it has two oxygen neighbors. 
### If it does, returns oxygen id's, anchor id, and oxygen's hydrogen id

function identify_carboxyl(crystal::Crystal, a::Int64)
   
    species = crystal.atoms.species[a]    #Identify species of selected atom
    nbs = neighbors(crystal.bonds, a)     #Identify neighbors of atom
    
    if species == :C                      #Identifying carboxylate starts with carbon
        oxygen_counter = 0
        oxygen_id = Int64[]
        X_id = 0
        H_id = 0
        for nb in nbs                     #Iterating through neighbors via identifying number
            species_nb = crystal.atoms.species[nb] #Get species of neighbor
            if species_nb == :O
                
                next_nbs = neighbors(crystal.bonds, nb)  #Get neighbors of oxygen (finding hydrogen)
                
                for next_nb in next_nbs
                    species_next_nb = crystal.atoms.species[next_nb]  #Get species of oxygen's neighbors
                    if species_next_nb == :H     #Identify hydrogen neighbor
                        H_id = next_nb
                    else
                    end
                end
                
                oxygen_counter += 1
                push!(oxygen_id, nb)         #Enter oxygen id into array
            else
                X_id = nb            #If neighbor of carboxylate carbon isn't oxygen, must be anchor
            end
        end
        
        if oxygen_counter == 2   #Assuming carboxylate carbon is only carbon with two oxygens bonded
        
            return true, oxygen_id, X_id, H_id
        else 
            return false, [], 0, 0
        end
    else
        return false, [], 0, 0
    end

end

identify_carboxyl (generic function with 1 method)

In [157]:
identify_carboxyl(crystal, 2)

(true, [1, 3], 4, 5)

In [158]:
#

keep = [true for i = 1:crystal.atoms.n]

for a = 1:crystal.atoms.n
       
   

    is_carboxyl, oxygen_ids, X_id, H_id = identify_carboxyl(crystal, a)
    
    if is_carboxyl == true 
    
        crystal.atoms.species[X_id] = :X
        @assert length(oxygen_ids) == 2
        keep[oxygen_ids] .= false
        keep[a] = false
        keep[H_id] = false
        
    end


end

In [159]:
for ed in edges(crystal.bonds)
   
    println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = crystal.atoms.species[i]
    species_j = crystal.atoms.species[j]
    println("atom ", species_i, " is connected to ", species_j)
    

end

Edge 1 => 2
atom O is connected to C
Edge 1 => 5
atom O is connected to H
Edge 2 => 3
atom C is connected to O
Edge 2 => 4
atom C is connected to X
Edge 4 => 6
atom X is connected to C
Edge 4 => 9
atom X is connected to N
Edge 4 => 16
atom X is connected to H
Edge 6 => 7
atom C is connected to C
Edge 6 => 13
atom C is connected to H
Edge 6 => 15
atom C is connected to H
Edge 7 => 8
atom C is connected to N
Edge 7 => 10
atom C is connected to C
Edge 7 => 14
atom C is connected to H
Edge 8 => 9
atom N is connected to N
Edge 8 => 12
atom N is connected to H
Edge 9 => 11
atom N is connected to H
Edge 10 => 17
atom C is connected to C
Edge 10 => 21
atom C is connected to C
Edge 17 => 18
atom C is connected to C
Edge 17 => 24
atom C is connected to H
Edge 18 => 19
atom C is connected to X
Edge 18 => 23
atom C is connected to H
Edge 19 => 20
atom X is connected to C
Edge 19 => 22
atom X is connected to C
Edge 20 => 21
atom C is connected to C
Edge 20 => 26
atom C is connected to H
Edge 21 => 

In [160]:
#Make a boolean array to keep atoms. 
crystal.atoms[keep]

Atoms{Frac}(21, [:X, :C, :C, :N, :N, :C, :H, :H, :H, :H  …  :H, :C, :C, :X, :C, :C, :H, :H, :H, :H], Frac([0.026193333333333315 0.07628666666666664 … 0.18547999999999998 0.22455999999999993; 0.34368 0.4270933333333333 … 0.5731733333333333 0.7288666666666667; -0.01016 0.015233333333333333 … -0.05088666666666666 -0.02521333333333333]))

In [161]:
tobacco_crystal = crystal.atoms[keep]

Atoms{Frac}(21, [:X, :C, :C, :N, :N, :C, :H, :H, :H, :H  …  :H, :C, :C, :X, :C, :C, :H, :H, :H, :H], Frac([0.026193333333333315 0.07628666666666664 … 0.18547999999999998 0.22455999999999993; 0.34368 0.4270933333333333 … 0.5731733333333333 0.7288666666666667; -0.01016 0.015233333333333333 … -0.05088666666666666 -0.02521333333333333]))

In [162]:
write_cif(crystal, "tobacco_crystal.cif")

In [163]:
crystal

Name: NewTest.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 15.000000 Å. b = 15.000000 Å, c = 15.000000 Å
	Volume of unit cell: 3375.000000 Å³

	# atoms = 29
	# charges = 0
	chemical formula: Dict(:N => 2,:H => 12,:X => 2,:O => 4,:C => 9)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [164]:
write_cif(crystal, "tobacco_crystal.cif")

In [165]:
#infer_bonds!(tobacco_crystal, true)
#remove_bonds!(tobacco_crystal)

In [166]:
tobacco_crystal = Crystal("tobacco_crystal_2.cif", box, atoms[keep], Charges{Frac}(0))

Name: tobacco_crystal_2.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 15.000000 Å. b = 15.000000 Å, c = 15.000000 Å
	Volume of unit cell: 3375.000000 Å³

	# atoms = 21
	# charges = 0
	chemical formula: Dict(:N => 2,:H => 10,:X => 2,:C => 7)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [167]:
xf_cm = sum(tobacco_crystal.atoms.coords.xf, dims = 2)


3×1 Array{Float64,2}:
  0.7675666666666661
 11.40856
 -0.29290666666666665

In [168]:
tobacco_crystal.atoms.coords.xf

3×21 Array{Float64,2}:
  0.0261933  0.0762867   0.0240067  …  -0.0904067    0.18548     0.22456
  0.34368    0.427093    0.49202        0.63422      0.573173    0.728867
 -0.01016    0.0152333  -0.04132        0.00708667  -0.0508867  -0.0252133

In [169]:
frctn_xf_cm = xf_cm ./ tobacco_crystal.atoms.n

3×1 Array{Float64,2}:
  0.036550793650793625
  0.5432647619047619
 -0.013947936507936506

In [170]:
new_coords = tobacco_crystal.atoms.coords.xf .- frctn_xf_cm

3×21 Array{Float64,2}:
 -0.0103575    0.0397359  -0.0125441  …  -0.126957    0.148929    0.188009
 -0.199585    -0.116171   -0.0512448      0.0909552   0.0299086   0.185602
  0.00378794   0.0291813  -0.0273721      0.0210346  -0.0369387  -0.0112654

In [171]:
tobacco_crystal.atoms.coords.xf .= mod.(tobacco_crystal.atoms.coords.xf, 1)

3×21 Array{Float64,2}:
 0.0261933  0.0762867  0.0240067  0.930893  …  0.909593    0.18548   0.22456
 0.34368    0.427093   0.49202    0.464933     0.63422     0.573173  0.728867
 0.98984    0.0152333  0.95868    0.973773     0.00708667  0.949113  0.974787

In [172]:
tobacco_crystal.atoms.coords.xf .= new_coords

3×21 Array{Float64,2}:
 -0.0103575    0.0397359  -0.0125441  …  -0.126957    0.148929    0.188009
 -0.199585    -0.116171   -0.0512448      0.0909552   0.0299086   0.185602
  0.00378794   0.0291813  -0.0273721      0.0210346  -0.0369387  -0.0112654

In [173]:
tobacco_crystal.atoms

Atoms{Frac}(21, [:X, :C, :C, :N, :N, :C, :H, :H, :H, :H  …  :H, :C, :C, :X, :C, :C, :H, :H, :H, :H], Frac([-0.01035746031746031 0.039735873015873016 … 0.14892920634920637 0.18800920634920631; -0.19958476190476193 -0.11617142857142859 … 0.029908571428571396 0.18560190476190475; 0.003787936507936506 0.02918126984126984 … -0.03693873015873016 -0.011265396825396824]))

In [174]:
function write_cif_Kai(crystal::Crystal, filename::AbstractString; fractional_coords::Bool=true,
		   number_atoms::Bool=true)
    if has_charges(crystal)
        if crystal.atoms.n != crystal.charges.n
            error("write_cif assumes equal numbers of Charges and Atoms (or zero charges)")
        end
        if ! isapprox(crystal.charges.coords, crystal.atoms.coords)
            error("write_cif needs coords of atoms and charges to correspond.")
        end
    end

    # TODO is this labeling necessary for the bonds, arthur?
    # create dictionary for tracking label numbers
    label_numbers = Dict{Symbol, Int}()
    for atom in crystal.atoms.species
        if !haskey(label_numbers, atom)
            label_numbers[atom] = 1
        end
    end

    # append ".cif" to filename if it doesn't already have the extension
    if ! occursin(".cif", filename)
        filename *= ".cif"
    end
    cif_file = open(filename, "w")
    # first line should be data_xtalname_PM
    if crystal.name == ""
        @printf(cif_file, "data_PM\n")
    else
        # don't include file extension!
        @printf(cif_file, "data_%s_PM\n", split(crystal.name, ".")[1])
    end

    @printf(cif_file, "_symmetry_space_group_name_H-M\t'%s'\n", crystal.symmetry.space_group)

    @printf(cif_file, "_cell_length_a\t%f\n", crystal.box.a)
    @printf(cif_file, "_cell_length_b\t%f\n", crystal.box.b)
    @printf(cif_file, "_cell_length_c\t%f\n", crystal.box.c)

    @printf(cif_file, "_cell_angle_alpha\t%f\n", crystal.box.α * 180.0 / pi)
    @printf(cif_file, "_cell_angle_beta\t%f\n", crystal.box.β * 180.0 / pi)
    @printf(cif_file, "_cell_angle_gamma\t%f\n", crystal.box.γ * 180.0 / pi)

    @printf(cif_file, "_symmetry_Int_Tables_number 1\n\n")
    @printf(cif_file, "loop_\n_symmetry_equiv_pos_as_xyz\n")
    for i in 1:size(crystal.symmetry.operations, 2)
        @printf(cif_file, "'%s,%s,%s'\n", crystal.symmetry.operations[:, i]...)
    end
    @printf(cif_file, "\n")

    @printf(cif_file, "loop_\n_atom_site_label\n_atom_site_type_symbol\n")
    if fractional_coords
        @printf(cif_file, "_atom_site_fract_x\n_atom_site_fract_y\n_atom_site_fract_z\n")
    else
        @printf(cif_file, "_atom_site_Cartn_x\n_atom_site_Cartn_y\n_atom_site_Cartn_z\n")
    end
    high_precision_charges = false # if, for neutrality, need high-precision charges
    if has_charges(crystal)
        @printf(cif_file, "_atom_site_charge\n")
        # if crystal will not be charge neutral to a 1e-5 tolerance when loading it
        #    into PorousMaterials.jl, then write higher-precision charges
        if abs(sum(round.(crystal.charges.q, digits=6))) > NET_CHARGE_TOL
            @info "writing high-precision charges for " * filename * ".\n"
            high_precision_charges = true
        end
    end

    idx_to_label = Array{AbstractString, 1}(undef, crystal.atoms.n)
    for i = 1:crystal.atoms.n
        # print label and type symbol
        @printf(cif_file, "%s\t%s\t", string(crystal.atoms.species[i]) *
                (number_atoms ? string(label_numbers[crystal.atoms.species[i]]) : ""),
                crystal.atoms.species[i])
        # store label for this atom idx
        idx_to_label[i] = string(crystal.atoms.species[i]) *
                    string(label_numbers[crystal.atoms.species[i]])
        # increment label
        label_numbers[crystal.atoms.species[i]] += 1
        if fractional_coords
            @printf(cif_file, "%f\t%f\t%f", crystal.atoms.coords.xf[:, i]...)
        else
            @printf(cif_file, "%f\t%f\t%f", (crystal.box.f_to_c * crystal.atoms.coords.xf[:, i])...)
        end
        if has_charges(crystal)
            if high_precision_charges
                @printf(cif_file, "\t%.10f\n", crystal.charges.q[i])
            else
                @printf(cif_file, "\t%f\n", crystal.charges.q[i])
            end
        else
            @printf(cif_file, "\n")
        end
        ## Kai Edit
        #if crystal.atoms.species[i] == :X
            
        #end
    end

    # only print bond information if it is in the crystal
    if ne(crystal.bonds) > 0
        if ! number_atoms
             error("must label atoms with numbers to write bond information.\n")
        end
        # print column names for bond information
        @printf(cif_file, "\nloop_\n_geom_bond_atom_site_label_1\n_geom_bond_atom_site_label_2\n_geom_bond_distance\n_ccdc_geom_bond_type\n")

        for edge in collect(edges(crystal.bonds))
            dxf = crystal.atoms.coords.xf[:, edge.src] - crystal.atoms.coords.xf[:, edge.dst]
            nearest_image!(dxf)
            @printf(cif_file, "%s\t%s\t%0.5f\n", idx_to_label[edge.src], idx_to_label[edge.dst],
                    norm(dxf))
        end
    end
    close(cif_file)
end


write_cif_Kai (generic function with 1 method)

In [181]:
write_cif_Kai(tobacco_crystal, "tobacco_crystal_2.cif")

LoadError: UndefVarError: norm not defined

In [182]:
infer_bonds!(tobacco_crystal, true)

LoadError: AssertionError: The crystal tobacco_crystal_2.cif already has bonds. Remove them with the `remove_bonds!` function before inferring new ones.

In [183]:
write_bond_information(tobacco_crystal)

Saving bond information for crystal tobacco_crystal_2.cif to C:\Users\kgeri\tobacco_crystal_2_bonds.vtk.


In [184]:
write_cif_Kai(tobacco_crystal, "tobacco_crystal_3.cif")

LoadError: UndefVarError: norm not defined

In [179]:
crystal.atoms.species_col

LoadError: type Atoms has no field species_col